In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re
import random
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

In [2]:
project_root_path = os.path.join('..', '..')
experiment_path = os.path.join(project_root_path, 'experiments', 'regression_08_01_2025')
results_path = os.path.join(experiment_path, 'results')

In [3]:
def load_results(csv_file):
    return pd.read_csv(csv_file, index_col=0)

result_files = [file for file in os.listdir(results_path) if file.endswith('.csv')]
supervised_files = [file for file in result_files if "_supervised" in file]
unsupervised_files = [file for file in result_files if "_unsupervised_" in file]

In [4]:
supervised_result = load_results(os.path.join(results_path, supervised_files[0]))

In [26]:
# Load supervised results
supervised_results_list = []

for file in tqdm(random.sample(supervised_files, 10)):
    parts = file.split('_')
    
    if len(parts) != 5:
        print("STOP! THERE IS A MISTAKE WITH GROUPING THE FILES. FILE NAME HAS PROBABLY CHANGED. TAKE CARE")
    else:
        curr_df = load_results(os.path.join(results_path, file))
        curr_df['Time series'] = curr_df.index.to_series().apply(lambda idx: '.'.join(idx.rsplit('.', maxsplit=1)[:-1]))
        curr_mean_df = curr_df.groupby('Time series').agg('mean')
        curr_mean_df['Regressor name'] = parts[0]
        # curr_mean_df['Regressor type'] = parts[1]
        curr_mean_df['Window size'] = parts[2]
        curr_mean_df['Detector'] = parts[3]
        supervised_results_list.append(curr_mean_df)

supervised_results_df = pd.concat(supervised_results_list)
supervised_results_df.reset_index(inplace=True)

regressor_df = supervised_results_df[['Regressor name', 'Window size']].drop_duplicates().reset_index(drop=True)

100%|███████████████████████████████████████████████| 10/10 [00:13<00:00,  1.38s/it]


In [27]:
# supervised_results_df['Selected detector'] = supervised_results_df.idxmax(axis=1)
# supervised_results_df['Selected VUS-PR'] = supervised_results_df.apply(lambda row: detectors_df.loc[row.name][row['Selected detector']], axis=1)
# supervised_results_df['Oracle'] = results.apply(lambda row: detectors_df.loc[row.name].max(), axis=1)
# supervised_results_df['Dataset'] = results.index.to_series().apply(lambda idx: idx.split('/', maxsplit=1)[0])
# results_melted = results[['Selected VUS-PR', 'Oracle']].melt(var_name='Metric', value_name='Value')

supervised_results_df

,Time series,label,y_pred,Regressor name,Window size,Detector
0,Daphnet/S01R02E0.test.csv@4.out,0.055895,0.214084,XGBRegressor,64,IFOREST
1,Daphnet/S02R01E0.test.csv@3.out,0.436835,0.270518,XGBRegressor,64,IFOREST
2,Daphnet/S03R01E1.test.csv@2.out,0.481405,0.246379,XGBRegressor,64,IFOREST
3,Daphnet/S03R01E1.test.csv@4.out,0.519852,0.242185,XGBRegressor,64,IFOREST
4,Daphnet/S03R01E1.test.csv@7.out,0.509005,0.282799,XGBRegressor,64,IFOREST
...,...,...,...,...,...,...
4965,YAHOO/Yahoo_A2synthetic_79_data.out,0.015767,0.129981,XGBRegressor,128,CNN
4966,YAHOO/Yahoo_A2synthetic_7_data.out,0.593977,0.159346,XGBRegressor,128,CNN
4967,YAHOO/Yahoo_A2synthetic_89_data.out,0.686305,0.430663,XGBRegressor,128,CNN
4968,YAHOO/Yahoo_A2synthetic_91_data.out,0.034331,0.133874,XGBRegressor,128,CNN
